In [1]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

/home/sebas/Documentos/bootcamp/assignment1/.env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
def residual_block(inputs, input_filters, filters,
        downsample=False, training=True, reg=0.0001):
    
    net = tf.layers.conv2d(inputs=inputs, filters=filters[0],
        kernel_size=[1, 1], padding="same", activation=None,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(reg)
    )
    net = tf.nn.elu(net)

    if downsample:
        s = 2
    else:
        s = 1
        
    net = tf.layers.conv2d(inputs=net, filters=filters[0],
        kernel_size=[3, 3], strides=(s, s), padding='same', activation=None,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(reg)
    )
    net = tf.nn.elu(net)
    
    net = tf.layers.conv2d(inputs=net, filters=filters[1],
        kernel_size=[1, 1], padding="same", activation=None,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(reg)
    )
    net = tf.nn.elu(net)
   
    if downsample:
        skipped_inputs = tf.layers.conv2d(inputs=inputs,
            filters=filters[1], kernel_size=[3, 3], strides=(s, s),
            padding='same', activation=None
        )

    elif filters[1] != input_filters:
        skipped_inputs = tf.layers.conv2d(inputs=inputs,
            filters=filters[1], kernel_size=[1, 1], strides=(1, 1),
            padding='same', activation=None,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(reg)
        )

    else:
        skipped_inputs = inputs

    net = net + skipped_inputs
    
    return tf.nn.elu(net)
    

In [33]:
def residual_block_stage(inputs, num_blocks, input_filters, filters,
        downsample=False, training=True, reg=0.0001):
    
    feature_map = inputs
    previous_filters = input_filters
    downsample_ = downsample
 
    for i in range(num_blocks):

        if (downsample and i == 0):
            downsample_ = True

        feature_map = residual_block(feature_map, previous_filters,
            filters, downsample_, training, reg=reg
        )
        previous_filters = tf.shape(feature_map)[-1]

        downsample_ = False

    return feature_map

In [34]:
def _model(inputs, stages, num_classes, training, reg, dropout_rate):
    
    # First convolution
    net = tf.layers.conv2d(inputs=inputs, filters= 64,
        kernel_size=[5, 5], padding="same", activation=None
    )

    net = tf.nn.elu(net)
    
    input_filters = 64
    filters = (64, 256)
    downsample = False
    
    # RestNet structure
    for num_blocks in stages:
        net = residual_block_stage(inputs=net, num_blocks=num_blocks,
            input_filters=input_filters, filters=filters,
            downsample=downsample, training=training
        )
        
        downsample = True
        input_filters = filters[1]
        filters = (filters[0]*2, filters[1]*2)
        
        print(net.shape)
        
    
    # Reduce mean
    net = tf.reduce_mean(net, axis=[1,2])

    # Extra dense layer to add dropout
    if dropout_rate > 0:
        net = tf.layers.dropout(net, rate=dropout_rate, training=training)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=net, units=num_classes,
        #kernel_regularizer=tf.contrib.layers.l2_regularizer(reg)
    )

    return logits

In [35]:
inputs = tf.convert_to_tensor(np.random.randn(5,255,255,3))

In [36]:
logits = _model(inputs,stages=[3,4,6,3], num_classes=100, training=None, reg=0.0001, dropout_rate=0.2)

(5, 255, 255, 256)
(5, 255, 255, 256)
(5, 255, 255, 256)
(5, 255, 255, 256)
(5, 128, 128, 512)
(5, 128, 128, 512)
(5, 128, 128, 512)
(5, 128, 128, 512)
(5, 128, 128, 512)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 64, 64, 1024)
(5, 32, 32, 2048)
(5, 32, 32, 2048)
(5, 32, 32, 2048)
(5, 32, 32, 2048)


In [37]:
logits.shape

TensorShape([Dimension(5), Dimension(100)])